In [1]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
import pymorphy2
import re
import string
from tqdm import tqdm
from collections import Counter
import numpy as np
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora
from datetime import datetime

tqdm.pandas()
nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

from multiprocessing import Pool
morph = pymorphy2.MorphAnalyzer()

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mchelushkin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Подготовка данных

In [2]:
commentsDF = pd.read_csv('data/commentsRAW.csv', usecols=['id', 'date', 'text'])
answersDF = pd.read_csv('data/answersRAW.csv', usecols=['id', 'date', 'text'])
commentsDF = commentsDF[commentsDF.text.notna()]
answersDF = answersDF[answersDF.text.notna()]

In [3]:
allCommentsDF = commentsDF.append(answersDF, ignore_index=True)
allCommentsDF.shape

(1433391, 3)

In [ ]:
# sortedDF = allCommentsDF.sort_values(by=['date'])

In [4]:
punctuation = string.punctuation + '…«»—–'
def clean_text(text):
# Make text lowercase
    text = text.lower()
# remove text in square brackets
    text = re.sub(r'\[.*?\]', ' ', text)
# remove urls
    text = re.sub(r"http\S+", ' ', text)
# remove punctuation   
    text = re.sub(r'[%s]' % re.escape(punctuation), ' ', text)
# remove numbers
    text = re.sub(r'[0-9]', ' ', text)
# remove non letters
    text = re.sub(r'[^\w\s]', '', text)
# remove dublicate spaces
    text = re.sub('\s\s+', ' ', text)
    return text
comments_df_clean = pd.DataFrame(allCommentsDF.text.progress_apply(lambda x: clean_text(x)))

100%|██████████| 1433391/1433391 [00:30<00:00, 46944.88it/s]


In [5]:
%%time

stopwords_list = stopwords.words('russian') + stopwords.words('english')
stopwords_list.append('это')
stopwords_list.append('всё')
stopwords_list.append('ещё')
stopwords_list.append('весь')
stopwords_list.append('человек')
stopwords_list.append('свой')
stopwords_list.append('который')
stopwords_list.append('мочь')
stopwords_list.append('')
stopwords_list.append('p')
stopwords_list.append('c')
stopwords_list.append('de')
stopwords_list.append('b')

def lemmatize(text, lemmer = morph, stopwords = stopwords_list):
    words = text.split(' ')
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if len(w) >= 3 and not w in stopwords]

with Pool(processes = 4) as pool:
    tmp = pool.starmap(lemmatize, zip(comments_df_clean.text))

CPU times: user 9.73 s, sys: 1.34 s, total: 11.1 s
Wall time: 5min 42s


In [4]:
import json
with open('LDA/lemmatized.json', 'r') as f:
    lemmatized = json.loads(f.read())

In [5]:
comments_lemmatized_df = pd.DataFrame({'text' : lemmatized, 'date' : allCommentsDF.date})

In [6]:
def wordsCount(texts):
    count = Counter()
    for words in texts:
        for word in words:
            count[word] += 1
    return count

def removeNotCommonWords(text, word_counter):
    return [word for word in text if word_counter[word] > 10]

def func(df):
    word_counter = wordsCount(df.text.values.tolist())
    
    df.text = df.text.apply(lambda x: removeNotCommonWords(x, word_counter))
    df.text = df.text.apply(lambda x: ' '.join(x))
    df.text = df.text.apply(lambda x: x.split())
    df = df[df.text.apply(lambda x: len(x) >= 40 and len(x) <= 300)]
    return df

In [7]:
clean_texts = func(comments_lemmatized_df)

In [28]:
from gensim import corpora
dictionary = corpora.Dictionary(clean_texts.text)

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
corpus = [dictionary.doc2bow(text) for text in clean_texts.text]

In [9]:
texts = clean_texts.text.apply(lambda x: ' '.join(x)).to_list()

35096

# Обучение модели BTM

In [13]:
import bitermplus as btm

In [14]:
X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
tf = np.array(X.sum(axis=0)).ravel()
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
docs_lens = list(map(len, docs_vec))

In [19]:
biterms = btm.get_biterms(docs_vec)

In [20]:
model = btm.BTM(
    X, vocabulary, seed=42, T=25, W=vocabulary.size, M=20, alpha=2, beta=0.01)

In [21]:
model.fit(biterms, iterations=50)

100%|██████████| 50/50 [05:30<00:00,  6.60s/it]


In [22]:
top_words = btm.get_top_topic_words(
    model,
    words_num=10)

In [23]:
top_words

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic15,topic16,topic17,topic18,topic19,topic20,topic21,topic22,topic23,topic24
0,бог,просто,covid,дом,вакцина,год,маска,ребёнок,число,ваш,...,тест,российский,страна,коронавирус,вирус,режим,сделать,деньга,заболевание,россия
1,сказать,деньга,вирус,сидеть,год,смерть,вирус,врач,коронавирус,говорить,...,врач,закон,год,ответить,иммунитет,коронавирус,главное,работать,коронавирус,народ
2,год,год,источник,маска,вакцинация,болезнь,рука,школа,заболеть,просто,...,день,чрезвычайный,россия,комментарий,грипп,москва,просто,платить,covid,режим
3,земля,хороший,животное,ребёнок,препарат,ребёнок,носить,год,случай,жизнь,...,больница,право,сша,список,организм,карантин,каждый,работа,вирус,наш
4,мир,делать,коронавирус,работа,исследование,жизнь,перчатка,работать,страна,знать,...,сказать,ситуация,мир,информация,коронавирус,день,сидеть,год,респираторный,власть
5,аллах,большой,воз,ходить,прививка,заболевание,защита,хороший,умерший,твой,...,делать,гражданин,китай,вирус,болезнь,гражданин,страна,государство,инфекция,путин
6,день,жить,sars,магазин,коронавирус,здоровье,лицо,работа,год,хороший,...,дом,федерация,жить,новость,вакцина,мера,факт,налог,вызывать,против
7,говорить,очень,иметь,карантин,первый,умереть,воздух,наш,россия,слово,...,сделать,режим,путин,писать,иметь,город,бог,зарплата,последний,страна
8,время,работать,cov,работать,страна,умирать,больной,учитель,смертность,сказать,...,поликлиника,статья,наш,деятельность,очень,случай,масло,месяц,синдром,война
9,наш,наш,исследование,жить,спутник,случай,магазин,медицинский,пневмония,хотеть,...,анализ,являться,европа,смысл,просто,также,уничтожить,рубль,открытый,год


In [30]:
topics = []
for topic in top_words.columns:
    topics.append(top_words[topic].to_list())

cm = CoherenceModel(topics=topics, texts=clean_texts.text, corpus=corpus, dictionary=dictionary, coherence='c_v')
cm.get_coherence()

0.5621970450155179

In [16]:
import pickle as pkl
# Saving
with open("model.pkl", "wb") as file:
    pkl.dump(model, file)

# # Loading
with open("model.pkl", "rb") as file:
    model = pkl.load(file)
